In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/youtube-video-ads/all_videos_topics.csv').drop(columns='Unnamed: 0')
df['link'] = '/kaggle/input/youtube-video-ads/data/' + df['class'] + '/' + df['link'] + '_' + df['class'] + '.mp4'

df.head()

,link,class
0,/kaggle/input/youtube-video-ads/data/Alcohol/1...,Alcohol
1,/kaggle/input/youtube-video-ads/data/Alcohol/2...,Alcohol
2,/kaggle/input/youtube-video-ads/data/Alcohol/2...,Alcohol
3,/kaggle/input/youtube-video-ads/data/Alcohol/3...,Alcohol
4,/kaggle/input/youtube-video-ads/data/Alcohol/4...,Alcohol


In [ ]:
! pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 47.6 MB/s eta 0:00:00:00:0100:01


In [ ]:
import av
import torch
import numpy as np

from transformers import AutoProcessor, AutoModel

np.random.seed(0)


def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


file_path = '/kaggle/input/youtube-video-ads/data/Alcohol/2S2VUCdbhg0_Alcohol.mp4'
container = av.open(file_path)

indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")

inputs = processor(videos=list(video), return_tensors="pt")

video_features = model.get_video_features(**inputs)

In [ ]:
video_features # можно построить для всех видосов и учиться на этом, пока лень

In [2]:
cats = ['Alcohol',
 'Charities',
 'animal_right',
 'baby',
 'beauty',
 'cars',
 'chips',
 'chocolate',
 'cleaning',
 'clothing',
 'coffee',
 'domestic_violence',
 'education',
 'electronics',
 'environment',
 'financial',
 'gambling',
 'game',
 'healthcare',
 'home_appliance',
 'home_improvement',
 'human_right',
 'media',
 'other_service',
 'petfood',
 'phone_tv_internet_providers',
 'political',
 'restaurant',
 'safety',
 'seasoning',
 'security',
 'self_esteem',
 'shopping',
 'smoking_alcohol_abuse',
 'soda',
 'software',
 'sports',
 'travel']

In [7]:
import av
import torch
import numpy as np

from transformers import AutoProcessor, AutoModel

np.random.seed(0)


def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


file_path = '/kaggle/input/youtube-video-ads/data/domestic_violence/RzDr18UYO18_domestic_violence.mp4'
container = av.open(file_path)

indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

processor = AutoProcessor.from_pretrained("microsoft/xclip-large-patch14") # microsoft/xclip-base-patch32 будет значительно быстрее, но хуже
model = AutoModel.from_pretrained("microsoft/xclip-large-patch14")

inputs = processor(
    text=cats,
    videos=list(video),
    return_tensors="pt",
    padding=True,
)

with torch.no_grad():
    outputs = model(**inputs)

logits_per_video = outputs.logits_per_video
probs = logits_per_video.softmax(dim=1)
print(probs)

preprocessor_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/8.90k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Unused or unrecognized kwargs: padding.


tensor([[4.5602e-03, 4.7356e-03, 1.5340e-03, 4.8192e-03, 9.3617e-03, 9.5480e-04,
         7.3969e-03, 3.5365e-04, 4.7143e-03, 4.3138e-04, 3.2520e-04, 9.0296e-01,
         4.7496e-03, 1.7981e-04, 5.2141e-04, 1.5915e-03, 6.2292e-03, 8.0079e-04,
         5.4279e-04, 1.0193e-03, 1.5927e-03, 3.4510e-04, 1.6978e-04, 9.3953e-04,
         1.4380e-03, 1.1267e-03, 1.2318e-03, 5.5164e-04, 1.6341e-02, 1.5449e-03,
         8.6232e-03, 1.7883e-03, 1.7721e-04, 3.4561e-03, 3.0385e-04, 9.1593e-04,
         1.2136e-03, 4.6363e-04]])


In [8]:
cats[int(np.argmax(probs))] # в целом хорошо работает из коробки, но нужно меньше классов, на таком количестве посос

'domestic_violence'

In [9]:
dct = {x: 0 for x in cats}

for i in range(len(cats)):
    dct[cats[i]] = float(probs[0][i])

In [10]:
dct

{'Alcohol': 0.0045602088794112206,
 'Charities': 0.004735603928565979,
 'animal_right': 0.001533995382487774,
 'baby': 0.0048192329704761505,
 'beauty': 0.009361662901937962,
 'cars': 0.00095479900483042,
 'chips': 0.00739690475165844,
 'chocolate': 0.0003536523145157844,
 'cleaning': 0.004714334849268198,
 'clothing': 0.0004313796234782785,
 'coffee': 0.00032519601518288255,
 'domestic_violence': 0.902955949306488,
 'education': 0.004749556537717581,
 'electronics': 0.00017981327255256474,
 'environment': 0.0005214101984165609,
 'financial': 0.001591507694683969,
 'gambling': 0.006229192018508911,
 'game': 0.0008007889846339822,
 'healthcare': 0.0005427923169918358,
 'home_appliance': 0.0010192592162638903,
 'home_improvement': 0.0015927336644381285,
 'human_right': 0.00034509843681007624,
 'media': 0.00016978185158222914,
 'other_service': 0.0009395282831974328,
 'petfood': 0.0014379883650690317,
 'phone_tv_internet_providers': 0.001126699848100543,
 'political': 0.001231755595654249

In [3]:
! git clone https://github.com/PKU-YuanGroup/Video-LLaVA

Cloning into 'Video-LLaVA'...
remote: Enumerating objects: 951, done.
remote: Counting objects: 100% (464/464), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 951 (delta 379), reused 278 (delta 278), pack-reused 487
Receiving objects: 100% (951/951), 115.32 MiB | 55.44 MiB/s, done.
Resolving deltas: 100% (519/519), done.


In [4]:
cd Video-LLaVA

/kaggle/working/Video-LLaVA


In [5]:
! ls

LICENSE    TRAIN_AND_VALIDATE.md  pyproject.toml  videollava
README.md  assets		  scripts


In [6]:
! pip install --upgrade pip
! pip install -e .
! pip install -e ".[train]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Obtaining file:///kaggle/working/Video-LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while

In [7]:
! pip install flash-attn --no-build-isolation
! pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=121687847 sha256=7544f11c6e8f86ed1b737b01bbe4fb1ccfad155dea5ff2d287699d04e6728459
  Stored in directory: /root/.cache/pip/wheels/9b/5b/2b/dea8af4e954161c49ef1941938afcd91bb93689371ed12a226
Successfully built flash-attn
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-req-build-bgydfamv
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-req-build-bgydfamv
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b9

In [8]:
import torch
from transformers import pipeline

from videollava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from videollava.conversation import conv_templates, SeparatorStyle
from videollava.model.builder import load_pretrained_model
from videollava.utils import disable_torch_init
from videollava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

2024-05-15 11:52:48.248574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 11:52:48.248674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 11:52:48.390825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[2024-05-15 11:52:58,252] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [9]:
disable_torch_init() # запускать только один раз

In [10]:
model_path = 'LanguageBind/Video-LLaVA-7B'
cache_dir = 'cache_dir'
device = 'cuda'

load_4bit, load_8bit = True, False
model_name = get_model_name_from_path(model_path)
tokenizer, model, processor, _ = load_pretrained_model(model_path, 
                                                       None, 
                                                       model_name, 
                                                       load_8bit, 
                                                       load_4bit, 
                                                       device=device, 
                                                       cache_dir=cache_dir)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/150k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at LanguageBind/Video-LLaVA-7B were not used when initializing LlavaLlamaForCausalLM: ['model.video_tower.video_tower.encoder.layers.12.self_attn.q_proj.weight', 'model.video_tower.video_tower.encoder.layers.5.mlp.fc1.weight', 'model.image_tower.image_tower.encoder.layers.7.layer_norm2.weight', 'model.image_tower.image_tower.encoder.layers.23.self_attn.k_proj.weight', 'model.video_tower.video_tower.encoder.layers.23.mlp.fc1.bias', 'model.image_tower.image_tower.encoder.layers.23.layer_norm2.weight', 'model.video_tower.video_tower.encoder.layers.7.self_attn.out_proj.bias', 'model.video_tower.video_tower.encoder.layers.18.layer_norm2.weight', 'model.video_tower.video_tower.encoder.layers.5.self_attn.out_proj.bias', 'model.video_tower.video_tower.encoder.layers.9.temporal_attn.out_proj.bias', 'model.video_tower.video_tower.encoder.layers.6.self_attn.out_proj.bias', 'model.image_tower.image_tower.encoder.layers.14.mlp.fc1.bias', 'model.image_tower.image

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

In [11]:
def getPrediction(video):
    inp = f'Which category from this list does the video belong to: {cats}?'
    
    video_processor = processor['video']
    conv_mode = "llava_v1"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles

    video_tensor = video_processor(video, return_tensors='pt')['pixel_values']
    if type(video_tensor) is list:
        tensor = [video.to(model.device, dtype=torch.float16) for video in video_tensor]
    else:
        tensor = video_tensor.to(model.device, dtype=torch.float16)
        
    inp = ' '.join([DEFAULT_IMAGE_TOKEN] * model.get_video_tower().config.num_frames) + '\n' + inp

    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)

    prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=tensor,
            do_sample=True,
            temperature=0.1,
            max_new_tokens=16,
            use_cache=True,
            stopping_criteria=[stopping_criteria])

    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()

    return outputs

In [13]:
video = '/kaggle/input/youtube-video-ads/data/domestic_violence/G9ZvxYcPJk8_domestic_violence.mp4'
getPrediction(video)

"The video belongs to the 'domestic_violence' category.</s>"